In [1]:
import numpy as np
import torch

In [2]:
a = [1., 2., 3.]

In [4]:
a[0]

1.0

In [7]:
a[2] = 3.
a

[1.0, 2.0, 3.0]

In [8]:
a = torch.ones(3)

In [9]:
a

tensor([1., 1., 1.])

In [10]:
float(a[1])

1.0

In [12]:
a[2] = 2.

In [13]:
a

tensor([1., 1., 2.])

In [14]:
points = torch.zeros(6)
points[0] = 4.
points[1] = 1.
points[2] = 5.
points[3] = 3.
points[4] = 2.
points[5] = 1.

In [15]:
points

tensor([4., 1., 5., 3., 2., 1.])

In [16]:
points = torch.tensor([4., 1., 5., 3., 2., 1.])

In [18]:
points

tensor([4., 1., 5., 3., 2., 1.])

In [19]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [21]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [22]:
points.shape

torch.Size([3, 2])

In [23]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [24]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [25]:
points[0, 1]

tensor(1.)

In [26]:
points[0][1]

tensor(1.)

In [27]:
points[0]

tensor([4., 1.])

In [28]:
some_list = list(range(6))
some_list[:]

[0, 1, 2, 3, 4, 5]

In [29]:
some_list[1:4]

[1, 2, 3]

In [30]:
some_list[1:]

[1, 2, 3, 4, 5]

In [31]:
some_list[:4]

[0, 1, 2, 3]

In [32]:
some_list[:-1]

[0, 1, 2, 3, 4]

In [34]:
some_list[1:4:2]

[1, 3]

In [35]:
points[None]

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

In [36]:
np_points = np.array([[4., 1,], [5., 3.], [2., 1.]])

In [37]:
np_points[None]

array([[[4., 1.],
        [5., 3.],
        [2., 1.]]])

In [38]:
from PIL import Image
from torchvision import transforms

preproscess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

img = Image.open("../../data/p1ch2/bobby.jpg")
img_t = preproscess(img)

In [39]:
img_t = torch.randn(3, 5, 5)
weight = torch.tensor([0.2126, 0.7152, 0.0722])

In [41]:
batch_t = torch.randn(2, 3, 5, 5)

In [42]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [47]:
test_mean = np.arange(1, 25).reshape(2, 3, 4)
test_mean

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]])

In [49]:
test_mean.mean(-1)

array([[ 2.5,  6.5, 10.5],
       [14.5, 18.5, 22.5]])

In [51]:
unsqueezed_weights = weight.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_wighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [52]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weight)
batch_gray_wighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weight)
batch_gray_wighted_fancy.shape

torch.Size([2, 5, 5])

In [53]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

C:\Users\NineLab\AppData\Local\Temp\ipykernel_23500\2371314847.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  C:\Users\builder\tkoch\workspace\pytorch\pytorch_1647970138273\work\c10/core/TensorImpl.h:1408.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [54]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named: ', img_named.shape, img_named.names)
print('batch named: ', batch_named.shape, batch_named.names)

img named:  torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named:  torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [55]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [56]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [57]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [58]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [61]:
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [62]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [63]:
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [64]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

In [65]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [67]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

In [68]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [69]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [70]:
points_storage = points.storage()
points_storage[0]

4.0

In [71]:
points.storage()[1]

1.0

In [72]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points_storage = points.storage()
points_storage[0] = 2.
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])